In [ ]:
# # Install Chromium and Chromedriver for Google Colab
# !apt-get update
# !apt-get install -y chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install selenium


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://cli.github.com/packages stable InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,576 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,569 kB]
Fetched 5,401 kB in 2s (2,907 kB/s)
Readi

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

# Set Chrome options
options = Options()
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--incognito")
options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 30)

# Open the Rotten Tomatoes browse page
driver.get("https://www.rottentomatoes.com/browse/movies_at_home")

# Load more movies (10 times)
for i in range(10):
    try:
        load_more = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-qa="dlp-load-more-button"]')))
        driver.execute_script("arguments[0].click();", load_more)
        print(f"Clicked Load More: {i + 1}")
        time.sleep(2)
    except Exception as e:
        print(f"Load more button not clickable at iteration {i+1}: {e}")
        break

# Let all content load
time.sleep(3)

# Get final page source
html = driver.page_source
driver.quit()

# Parse with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
cards = soup.select('[data-qa="discovery-media-list-item-caption"]')

# Extract movie data
movie_names = []
critic_scores = []
audience_scores = []

for card in cards:
    name = card.select_one('[data-qa="discovery-media-list-item-title"]')
    name = name.text.strip() if name else "N/A"

    critic = card.select_one('rt-text[slot="criticsScore"]')
    critic = critic.text.strip() if critic else "N/A"

    audience = card.select_one('rt-text[slot="audienceScore"]')
    audience = audience.text.strip() if audience else "N/A"

    movie_names.append(name)
    critic_scores.append(critic)
    audience_scores.append(audience)

# Create DataFrame
df = pd.DataFrame({
    "Movie Name": movie_names,
    "Tomatometer": critic_scores,
    "Audience Score": audience_scores
})

# Save to CSV
df.to_csv("rottentomatoes_movies.csv", index=False)
print(f"✅ Scraped {len(df)} movies. Data saved to rottentomatoes_movies.csv")

# Download the file in Colab
# from google.colab import files
# files.download("rottentomatoes_movies.csv")


Clicked Load More: 1
Clicked Load More: 2
Clicked Load More: 3
Clicked Load More: 4
Clicked Load More: 5
Clicked Load More: 6
Clicked Load More: 7
Clicked Load More: 8
Clicked Load More: 9
Clicked Load More: 10
✅ Scraped 328 movies. Data saved to rottentomatoes_movies.csv
